In [ ]:
%pip install requests
%pip install scikit-image
%pip install pillow
%pip install tqdm

In [ ]:
import requests
import urllib
import os
import threading
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from skimage.io import imread, imshow, imsave
from skimage.transform import rotate
from skimage.util import img_as_ubyte, random_noise
from skimage.filters import gaussian
from tqdm import tqdm
from PIL import Image

# Get Data From API

## Download Image Data
Download images for the top 10 breeds

### Organisation of images downloaded:
The data are downloaded to `./new_dog/[breed_name]/[breed_name]_xxx.jpg` where `[breed_name]` is the dog's breed and `xxx` is a 3-digit int from 000 to 999

In [ ]:
def download_imgs(breeds):
    img_count = 0

    # Not downloading if directory already exists
    if not os.path.exists('old_dog'):
        os.makedirs('old_dog')

        for breed in breeds:

            path_str = 'old_dog/' + breed
            if not os.path.exists(path_str):
                os.makedirs(path_str)

            print("Start for breed", breed)
            req_str = "https://dog.ceo/api/breed/{0}/images".format(breed)
            resp = requests.get(req_str).json()

            img_sources = resp['message']
            for i in range(len(img_sources)):

                source = img_sources[i]
                img_path = "{0}/{1}_{2:03}.jpg".format(path_str, breed, i)

                #urllib.request.urlretrieve(source, img_path)
                create_thread_for_download(source, img_path)

                img_count += 1

            print("Download all images for breed {0}.".format(breed))

        print("Success downloading all {0} images.".format(img_count))


# Use multithreading for download images
def download_by_req(source, img_path):
    urllib.request.urlretrieve(source, img_path)


def create_thread_for_download(source, img_path):
    download_thread = threading.Thread(target=download_by_req, args=(source, img_path))
    download_thread.start()

In [ ]:
breed_names_used = {'bulldog', 'hound', 'mountain', 'poodle', 'retriever', 'schnauzer', 'setter', 'sheepdog', 'spaniel', 'terrier'}
breed_names_dict = {}
# download_imgs(breed_names_used)
label = 1
for breed in breed_names_used:
    breed_names_dict[breed] = label
    label += 1
print(breed_names_dict)

## Generate csv containing image_name, label pairs

In [ ]:
def generate_csv(breed_names_dict):
    csv_file = open("dog_data.csv", "w")
    csv_file.write("image_name,label\n")

    for breed in breed_names_dict:
        path_str = 'old_dog/' + breed
        num_files = len([f for f in os.listdir(path_str) if os.path.isfile(os.path.join(path_str, f))])
        for i in range(0, num_files):
            img_path = "{0}/{1}_{2:03}.jpg".format(path_str, breed, i)
            csv_file.write("{0},{1}\n".format(img_path, breed_names_dict[breed]))

    csv_file.close()


In [ ]:
generate_csv(breed_names_dict)

# Resize all images
All images will be center-cropped and resized to dimension 224 x 224 (dimensions can be changed later)

In [ ]:
# resize all images in all folders to be of the same dimension
def resize_all_imgs(breed_names_dict):
    for breed in breed_names_dict:
        path_old_str = 'old_dog/' + breed
        path_str = 'new_dog/' + breed
        if not os.path.exists(path_str):
            os.makedirs(path_str)

        num_files = len([f for f in os.listdir(path_old_str) if os.path.isfile(os.path.join(path_old_str, f))])
        for i in range(0, num_files):
            img_path_old = "{0}/{1}_{2:03}.jpg".format(path_old_str, breed, i)
            img_path = "{0}/{1}_{2:03}.jpg".format(path_str, breed, i)

            img = Image.open(img_path_old)
            size = min(img.size)
            hmargin = (img.size[0] - size) // 2
            vmargin = (img.size[1] - size) // 2
            img = img.crop((hmargin, vmargin, size+hmargin, size+vmargin))
            img = img.resize((128, 128))
            img = img.convert('RGB')
            img.save(img_path)

In [ ]:
resize_all_imgs(breed_names_dict)

# Perform image augmentation to downloaded images

Augmentations include rotations, shifting, flipping, adding noise and using cutmix

In [ ]:
data = pd.read_csv("dog_data.csv")
data.head()
all_img = []
for image_path in tqdm(data['image_name']):
    image_path = image_path.replace('old', 'new')
    img = imread(image_path)
    all_img.append(np.array(img, dtype=np.uint8))

train_x = np.stack(all_img)
train_y = data['label'].values
train_x.shape, train_y.shape

In [ ]:
# decrease all elements of final_train_y by 1
train_y -= 1
train_y

In [ ]:
imshow(train_x[np.random.choice(train_x.shape[0])])

## Implement Cut Mix augmentation

In [ ]:
def rand_bbox(size, lamb):
    """ Generate random bounding box 
    Args:
        - size: [width, breadth] of the bounding box
        - lamb: (lambda) cut ratio parameter, sampled from Beta distribution
    Returns:
        - Bounding box
    """
    W = size[0]
    H = size[1]
    cut_rat = np.sqrt(1. - lamb)
    cut_w = np.int32(W * cut_rat)
    cut_h = np.int32(H * cut_rat)

    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

In [ ]:
def generate_cutmix_image(image_batch, image_batch_labels, beta):
    """ Generate a CutMix augmented image from a batch 
    Args:
        - image_batch: a batch of input images
        - image_batch_labels: labels corresponding to the image batch
        - beta: a parameter of Beta distribution.
    Returns:
        - CutMix image batch, updated labels
    """
    # generate mixed sample
    lam = np.random.beta(beta, beta)
    rand_index = np.random.permutation(len(image_batch))
    target_a = image_batch_labels
    target_b = image_batch_labels[rand_index]
    bbx1, bby1, bbx2, bby2 = rand_bbox(image_batch[0].shape, lam)
    image_batch_updated = image_batch.copy()
    image_batch_updated[:, bbx1:bbx2, bby1:bby2, :] = image_batch[rand_index, bbx1:bbx2, bby1:bby2, :]
    
    # adjust lambda to exactly match pixel ratio
    lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (image_batch.shape[1] * image_batch.shape[2]))
    label = target_a * lam + target_b * (1. - lam)
    
    return image_batch_updated, label

In [ ]:
# Read an image
image = train_x[0]
image.shape

In [ ]:
import cv2

# Crop a random bounding box
lamb = 0.4
size = image.shape[0], image.shape[1]
bbox = rand_bbox(size, lamb)

# Draw bounding box on the image
im = image.copy()
x1 = bbox[0]
y1 = bbox[1]
x2 = bbox[2]
y2 = bbox[3]
cv2.rectangle(im, (x1, y1), (x2, y2), (255, 0, 0), 3)
plt.imshow(im)
plt.title('Original image with random bounding box')
plt.show()

# Show cropped image
plt.imshow(image[y1:y2, x1:x2])
plt.title('Cropped image')
plt.show()

Check result of cutmix augmentation

In [ ]:
cut_mix_img, cut_mix_labels = generate_cutmix_image(train_x, train_y, 0.5)

In [ ]:
imshow(cut_mix_img[0])
print(cut_mix_labels[0])

In [ ]:
all_img = []
final_train_x = []
final_train_y = []

for i in tqdm(range(train_x.shape[0])):
    img, label = train_x[i], train_y[i]
    all_img.extend([
        img,
        img_as_ubyte(rotate(img, angle=45, mode='wrap')),
        np.fliplr(img),
        np.flipud(img),
        img_as_ubyte(random_noise(train_x[i],var=0.2**2))
    ])
    final_train_y.extend([train_y[i]] * 5)

final_train_x = np.stack(all_img)
final_train_y = np.array(final_train_y)
# final_train_x = np.concatenate([final_train_x, cut_mix_img], axis=0)
# final_train_y = np.concatenate([final_train_y, cut_mix_labels], axis=0)
final_train_x.shape, final_train_y.shape

see results of augmentation

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 5, figsize = (20, 20))
idx = np.random.choice(train_x.shape[0])
for i in range(5):
    label = train_y[idx]
    ax[i].imshow(final_train_x[idx*5 + i])
    ax[i].axis('off')

see results of cut mix

In [ ]:
# imshow(final_train_x[len(final_train_x) - 1])
# print(final_train_y[len(final_train_y) - 1])

#Save results as pt files

In [ ]:
import torch

tensors_data = torch.from_numpy(final_train_x).float()
tensors_data = tensors_data.permute(0, 3, 1, 2)
tensors_label = torch.from_numpy(final_train_y).long()
tensors_data.size(), tensors_label.size()

In [ ]:
tensors_data = tensors_data.contiguous()
tensors_data.is_contiguous()

In [ ]:
torch.save(tensors_data, 'data/aug_data_large.pt')
torch.save(tensors_label, 'data/aug_label_large.pt')